<a href="https://colab.research.google.com/github/annawilson123/pseudocode-to-python-sourcecode/blob/main/Pseudocode_to_python_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-794io_p_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-794io_p_
  Resolved https://github.com/huggingface/transformers to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, GPTQConfig
from trl import SFTTrainer
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token


def generate_train():
  data=load_dataset("iamtarun/python_code_instructions_18k_alpaca",streaming=True,split="train", download_mode="force_redownload")
  samp_total=10000
  valid_p=0.1
  samp_train=int(samp_total* (1-valid_p))
  count=0
  for sample in iter(data):
    if count>=samp_train:
      break
    org_p=sample['Algorithm']
    question=org_p.strip()
    ans_start=sample['output']
    answer=org_p.strip()
    nf= f'<s>[INST]{question} [/INST] ```python\n{answer}```</s>'
    output_token = tokenizer(nf)
    yield {'text': nf}
    count += 1

def generate_validation():
  data=load_dataset("iamtarun/python_code_instructions_18k_alpaca",streaming=True,split="train", download_mode="force_redownload")
  samp_total=10000
  valid_p=0.1
  samp_train=int(samp_total* (1-valid_p))
  count=0
  for sample in iter(data):
    if count<samp_train:
      count+=1
      continue
    if count>=samp_total:
      break
    org_p=sample['Algorithm']
    question=org_p.strip()
    ans_start=sample['output']
    answer=org_p.strip()
    nf= f'<s>[INST]{question} [/INST] ```python\n {answer}```</s>'
    output_token = tokenizer(nf)
    yield {'text': nf}
    count += 1

def finetune_mistral_7b():

  config_details=GPTQConfig(
    bits=4,
    disable_exllama=True,
    tokenizer=tokenizer
    )


  dev_map="auto"

  model = AutoModelForCausalLM.from_pretrained(
        "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
        quantization_config=config_details,
        device_map=dev_map,
        use_auth_token=True
    )

  print(model)
  model.config.use_cache=False
  model.config.pretraining_tp=1
  model.gradient_checkpointing_enable()
  model = prepare_model_for_kbit_training(model)
  model.config.window = 256

  peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=32,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    )


  model = get_peft_model(model, peft_config)

  training_arguments = TrainingArguments(
        output_dir= "./results_packing",
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        save_steps=50,
        max_grad_norm=0.3,
        weight_decay=0.01,
        learning_rate=2e-5,
        lr_scheduler_type="constant",
        evaluation_strategy="steps",
        logging_steps=50,
        num_train_epochs=1,
        max_steps=250,
        warmup_ratio=0.03,
        group_by_length=True,
        fp16=True,
        push_to_hub=True
    )

  train_generator = Dataset.from_generator(generate_train)
  val_generator = Dataset.from_generator(generate_validation)

  tokenizer.padding_side = "right"

  trainer = SFTTrainer(
        model=model,
        train_dataset=train_generator,
        eval_dataset=val_generator,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )

  trainer.train()
  trainer.push_to_hub()

if __name__ == "__main__":
     finetune_mistral_7b()

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,1.259200,0.989052


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
! pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.7 MB/s eta 0:00:00


In [ ]:
! pip install auto-gptq